In [1]:
def Metropolis_Hasting(A, B, H, x, y, sigma_0, sigma_r, sigma_q, prior_mean, prior_var, T, N):
  param = []
  param.append(np.random.normal(prior_mean, prior_var))

  energy_list = []

  f = np.tile(np.array(np.random.normal(param[0],1, (int(np.shape(A)[0]/3),1))), (1,T))
  phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(param[0]-prior_mean)
  x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
  phi += energy
  energy_list.append(phi)
  for j in range(1,N):
    x = np.zeros((3*len(UA),1))
    x[0:6,0] = np.transpose(UA)
    proposal = np.random.normal(param[j-1], 1)
    f = np.tile(np.array(np.random.normal(proposal,1, (int(np.shape(A)[0]/3),1))), (1,T))
    phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(proposal-prior_mean)
    x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
    phi += energy

    acceptance = min(1, np.exp(energy_list[j-1]-phi))
    uniform = np.random.uniform(0,1)
    if uniform <= acceptance:
      param.append(proposal)
      energy_list.append(phi)
    else:
      param.append(param[j-1])
      energy_list.append(energy_list[j-1])
  return param, energy_list

In [ ]:
N = 100
sigma_0 = 0.1
sigma_r = 0.1
sigma_q = 0.1
y = data

Q = sigma_q*np.identity(np.shape(A)[0])
R = sigma_r*np.identity(np.shape(y)[0])
H = np.hstack((np.identity(len(UA)), np.zeros((len(UA),len(UA))), np.zeros((len(UA),len(UA)))))

x = np.zeros((3*len(UA),1))
x[0:6,0] = np.transpose(UA)

P = [sigma_0*np.identity(np.shape(A)[0])]

prior_mean = 0.8
prior_var = 1

mean = [] ### parameter theta
mean.append(np.random.normal(prior_mean, prior_var))

energy = []
energy.append(0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(mean[0]-prior_mean))
for i in range(T):
  f = np.random.normal(mean[0],1, len(PA))
  x_predict = np.dot(A, x[:,i])+np.dot(B, f)
  P_predict = np.dot(A, np.dot(P[i], np.transpose(A))) + Q
  V = y[:,i] - np.dot(H, x_predict)
  S = np.dot(H, np.dot(P_predict, np.transpose(H))) + R
  K = np.dot(P_predict, np.dot(np.transpose(H), np.linalg.inv(S)))
  x_estimate = x_predict + np.dot(K, V)
  P_estimate = P_predict - np.dot(K, np.dot(S, np.transpose(K)))

  x = np.append(x, x_estimate.reshape(3*len(UA),1), axis=1)
  P.append(P_estimate)

  energy[0] += 0.5*np.log(np.linalg.det(2*np.pi*S))+0.5*np.dot(np.transpose(V), np.dot(np.linalg.inv(S), V))

for j in range(1,N):
  x = np.zeros((3*len(UA),1))
  x[0:6,0] = np.transpose(UA)
  P = [sigma_0*np.identity(np.shape(A)[0])]

  proposal = np.random.normal(mean[j-1], 1)
  phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(proposal-prior_mean)

  for i in range(T):
    f = np.random.normal(proposal,1, len(PA))
    x_predict = np.dot(A, x[:,i])+np.dot(B, f)
    P_predict = np.dot(A, np.dot(P[i], np.transpose(A))) + Q
    V = y[:,i] - np.dot(H, x_predict)
    S = np.dot(H, np.dot(P_predict, np.transpose(H))) + R
    K = np.dot(P_predict, np.dot(np.transpose(H), np.linalg.inv(S)))
    x_estimate = x_predict + np.dot(K, V)
    P_estimate = P_predict - np.dot(K, np.dot(S, np.transpose(K)))

    x = np.append(x, x_estimate.reshape(3*len(UA),1), axis=1)
    P.append(P_estimate)

    phi += 0.5*np.log(np.linalg.det(2*np.pi*S))+0.5*np.dot(np.transpose(V), np.dot(np.linalg.inv(S), V))

  acceptance = min(1, np.exp(energy[j-1]-phi))
  uniform = np.random.uniform(0,1)
  if uniform <= acceptance:
    mean.append(proposal)
    energy.append(phi)
  else:
    mean.append(mean[j-1])
    energy.append(energy[j-1])

In [ ]:
index = np.argmin(energy)
mean_MAP = mean[index]
print(mean_MAP)
plt.hist(mean)

In [ ]:
def Metropolis_Hasting(A, B, H, x, force_func, y, sigma_0, sigma_r, sigma_q, prior_mean, prior_var, T, N, UA):
  param = []
  param.append(np.random.normal(prior_mean, prior_var))

  energy_list = []

  f = force_func(param[0], T)
  phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(param[0]-prior_mean)
  x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
  phi += energy
  energy_list.append(phi)
  for j in range(1,N):
    x = np.zeros((18,1))
    x[0:6,0] = np.transpose(UA)
    proposal = np.random.normal(param[j-1], 1)
    f = force_func(param[j-1], T)
    phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(proposal-prior_mean)
    x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
    phi += energy

    acceptance = min(1, np.exp(energy_list[j-1]-phi))
    uniform = np.random.uniform(0,1)
    if uniform <= acceptance:
      param.append(proposal)
      energy_list.append(phi)
    else:
      param.append(param[j-1])
      energy_list.append(energy_list[j-1])
  return param, energy_list

In [ ]:
def Metropolis_Hasting(A, B, H, x, y, sigma_0, sigma_r, sigma_q, prior_mean, prior_var, proposal_var, T, N):
  param = []
  param.append(np.random.normal(prior_mean, prior_var))

  energy_list = []

  f = np.tile(np.array(np.random.normal(param[0],1, (int(np.shape(A)[0]/3),1))), (1,T))
  phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(param[0]-prior_mean)
  x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
  phi += energy
  energy_list.append(phi)
  for j in range(1,N):
    x = np.zeros((3*len(UA),1))
    x[0:6,0] = np.transpose(UA)
    proposal = np.random.normal(param[j-1], proposal_var)
    f = np.tile(np.array(np.random.normal(proposal,1, (int(np.shape(A)[0]/3),1))), (1,T))
    phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(proposal-prior_mean)
    x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
    phi += energy

    acceptance = np.exp(min(0, (energy_list[j-1]-phi)))
    uniform = np.random.uniform(0,1)
    if uniform <= acceptance:
      param.append(proposal)
      energy_list.append(phi)
    else:
      param.append(param[j-1])
      energy_list.append(energy_list[j-1])
  return param, energy_list

In [ ]:
def Metropolis_Hastings(A, B, H, x, force_func, y, sigma_0, sigma_r, sigma_q, prior_mean, prior_var, proposal_var, T, N):
  x_init = x
  X = []

  param = []
  param.append(np.random.normal(prior_mean, prior_var))

  energy_list = []

  f = force_func(param[0], T)
  phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(param[0]-prior_mean)
  x, energy = Kalman_filter(A, B, H, x_init, f, y, sigma_0, sigma_r, sigma_q, T)
  X.append(x)
  phi += energy
  energy_list.append(phi)
  for j in range(1,N):
    proposal = np.random.normal(param[j-1], proposal_var)
    f = force_func(param[j-1], T)
    phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(proposal-prior_mean)
    x, energy = Kalman_filter(A, B, H, x_init, f, y, sigma_0, sigma_r, sigma_q, T)
    X.append(x)
    phi += energy

    acceptance = np.exp(min(0, (energy_list[j-1]-phi)))
    uniform = np.random.uniform(0,1)
    if uniform <= acceptance:
      param.append(proposal)
      energy_list.append(phi)
    else:
      param.append(param[j-1])
      energy_list.append(energy_list[j-1])
  return param, energy_list, X

In [ ]:
# def Metropolis_Hastings(A, B, H, x, force_func, y, sigma_0, sigma_r, sigma_q, prior_mean, prior_var, T, N):
#   param = []
#   param.append(np.random.normal(prior_mean, prior_var))

#   energy_list = []

#   for j in range(N):
#     x = x[:,0]
#     if j == 0:
#       f = np.random.normal(param[j],1, (np.shape(A)[0],T))
#       phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(param[j]-prior_mean)
#       x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
#       phi += energy
#       energy_list.append(phi)
#     else:
#       proposal = np.random.normal(param[j-1], 1)
#       f = np.random.normal(proposal,1, (np.shape(A)[0],T))
#       phi = 0.5*np.log(2*np.pi*prior_var)+1/(2*prior_var**2)*(proposal-prior_mean)
#       x, energy = Kalman_filter(A, B, H, x, f, y, sigma_0, sigma_r, sigma_q, T)
#       phi += energy

#       acceptance = min(1, np.exp(energy_list[j-1]-phi))
#       uniform = np.random.uniform(0,1)
#       if uniform <= acceptance:
#         param.append(proposal)
#         energy_list.append(phi)
#       else:
#         param.append(param[j-1])
#         energy_list.append(energy_list[j-1])
#   return param